In [2]:
# Organizing Topic Modeling Results

In [1]:
import pandas as pd

In [48]:
# Loading topic modeling results
reviews = pd.read_csv(".//LDA/if_mallet_41topics_representation.tsv", sep = '\t')
reviews = reviews.drop("Keywords", axis = 1)
reviews = reviews.drop("Unnamed: 0", axis = 1)
reviews = reviews.drop("Topic_Perc_Contrib", axis = 1)
reviews['Topic_Num'] = reviews['Topic_Num'].astype(int)

In [50]:
# Reassembling topics
calorie_tracking = reviews.loc[reviews['Topic_Num'].isin([20, 9, 35])] # according to topic index number
calorie_tracking["sub_topic"] = 1

tracking_features = reviews.loc[reviews['Topic_Num'].isin([12, 23, 40])]
tracking_features["sub_topic"] = 2

food_intake = reviews.loc[reviews['Topic_Num'] == 11]
food_intake["sub_topic"] = 3

time_logging = reviews.loc[reviews['Topic_Num'] == 6]
time_logging["sub_topic"] = 4

reminders = reviews.loc[reviews['Topic_Num'] == 23]
reminders["sub_topic"] = 5

guided_schedule = reviews.loc[reviews['Topic_Num'].isin([1, 17])]
guided_schedule["sub_topic"] = 6

workout_routines = reviews.loc[reviews['Topic_Num'].isin([26, 38])]
workout_routines["sub_topic"] = 7

meal_recommendations = reviews.loc[reviews['Topic_Num'].isin([19, 25])]
meal_recommendations["sub_topic"] = 8

informational_features = reviews.loc[reviews['Topic_Num'] == 14]
informational_features["sub_topic"] = 9

paid_content = reviews.loc[reviews['Topic_Num'] == 3]
paid_content["sub_topic"] = 10

weight_loss = reviews.loc[reviews['Topic_Num'].isin([2, 4, 10, 13, 15])]
weight_loss["sub_topic"] = 11

changing_lifestyle = reviews.loc[reviews['Topic_Num'] == 5]
changing_lifestyle["sub_topic"] = 12

improve_health = reviews.loc[reviews['Topic_Num'] == 18]
improve_health["sub_topic"] = 13

easy_use = reviews.loc[reviews['Topic_Num'].isin([16, 28, 36])]
easy_use["sub_topic"] = 14

flexibility = reviews.loc[reviews['Topic_Num'] == 8]
flexibility["sub_topic"] = 15

customization = reviews.loc[reviews['Topic_Num'] == 27]
customization["sub_topic"] = 16

ui = reviews.loc[reviews['Topic_Num'] == 7]
ui["sub_topic"] = 17

aesthetics = reviews.loc[reviews['Topic_Num'] == 31]
aesthetics["sub_topic"] = 18

df = pd.concat([calorie_tracking, tracking_features, food_intake, time_logging, reminders, guided_schedule,
                workout_routines, meal_recommendations, informational_features, paid_content, weight_loss, 
                changing_lifestyle, improve_health, easy_use, flexibility, customization, ui, aesthetics])

df = df.reset_index()
df = df.drop("Topic_Num", axis = 1)
df = df.drop("index", axis = 1)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [66]:
df

,Text,sub_topic,compound_scores
0,I tried a few different apps but I kept comin...,1,0.9911
1,Dofasting has been an amazing tool to keep tra...,1,0.9311
2,Quick and super easy to help you keep track of...,1,0.9674
3,The Keto app is a really useful tool that hel...,1,0.9840
4,This great app This great app has everything I...,1,0.9001
...,...,...,...
30221,Very quick and easy to use. Perfect for keepi...,18,0.7823
30222,It’s a yes from me! How can I say no to such ...,18,0.8999
30223,IF for the first time ❤️ I depend on this app ...,18,0.7372
30224,This is a great app to track your fasts and s...,18,0.8718


In [55]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [63]:
# Retrieving compound scores of each review
compound_scores = []

for i in df["Text"]:
    compound_scores.append(analyser.polarity_scores(i).get("compound"))
    
df['compound_scores'] = compound_scores

In [74]:
# decide sentiment as positive, negative and neutral 
sentiment = []

for i in df['compound_scores']:
    if i >= 0.05 : 
        sentiment.append("Positive") 
    elif i <= -0.05 : 
        sentiment.append("Negative") 
    else : 
        sentiment.append("Neutral")

print(len(sentiment))
df['sentiment'] = sentiment

30226


In [84]:
# Organizing sentiment distribution
positives = []
negatives = []
neutrals = []
for i in range(1, 19):
    cluster = df[df.sub_topic == i]
    total = cluster.shape[0]
    positive = cluster[cluster.sentiment == "Positive"].shape[0]
    negative = cluster[cluster.sentiment == "Negative"].shape[0]
    neutral = cluster[cluster.sentiment == "Neutral"].shape[0]
    positives.append(round(positive/total*100, 2))
    negatives.append(round(negative/total*100, 2))
    neutrals.append(round(neutral/total*100, 2))

In [88]:
sentiment_df = pd.DataFrame()
sentiment_df["topic_index"] = range(1,19)
sentiment_df["positive"] = positives
sentiment_df["negative"] = negatives
sentiment_df["neutral"] = neutrals

In [90]:
sentiment_df.to_csv("IF_sentiment_percentage.csv")